In [2]:
%matplotlib notebook

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import queue
import json

In [3]:
# constants

GRID_RESOLUTION = 10**4 # higher res means smaller cells
CLUSTERED_DIAMETER = 100 # ZxZ clusters

MIN_LAT_IN_DATASET = 16.4078522359116
MIN_LNG_IN_DATASET = 49.112730179544
GRID_CORNER_PADDING = 100
CORNER_LAT_POS = math.floor(MIN_LAT_IN_DATASET * GRID_RESOLUTION) - GRID_CORNER_PADDING
CORNER_LNG_POS = math.floor(MIN_LNG_IN_DATASET * GRID_RESOLUTION) - GRID_CORNER_PADDING
GRID_SIZE = 3500

NOCLUSTERS = int(GRID_SIZE / CLUSTERED_DIAMETER)

PERCENT_ITEMS = 0.25

In [4]:
def linspaceExtend(route, num):
    newpoints = []
    for i in range(len(route)-1):
        point1 = route[i]
        point2 = route[i+1]
        
        lat1 = point1['lat']
        lat2 = point2['lat']
        lng1 = point1['lng']
        lng2 = point2['lng']
        
        lats = np.linspace(lat1, lat2, num=num)
        lngs = np.linspace(lng1, lng2, num=num)
        
        newpoints = newpoints + [{"lat":a, "lng":b} for (a,b) in zip(lats, lngs)]
    return newpoints

In [5]:
# read planned routes from file

planned_routes = []
with open('data/brno_cyclo_route_general_plan.csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=',')
    for row in csvreader:
        route_str = row['wkt']
        route_str = route_str.replace("LINESTRING (", "").replace(")", "")
        
        points = route_str.split(',')
        route = []
        for point in points:
            coords = point.split(' ')
            lat = float(coords[0])
            lng = float(coords[1])
            route.append({"lat":lat, "lng":lng})
            
        planned_routes.append(route)
        
#for route in planned_routes:
#   planned_points = planned_points + linspaceExtend(route, 10)
# planned_routes = list(map(lambda r: linspaceExtend(r, 10), planned_routes))

planned_points = [points for route in planned_routes for points in route]
    
# lats = list(map(lambda p: p['lat'], planned_points))
# lngs = list(map(lambda p: p['lng'], planned_points))

In [6]:
# grid matrix

class Grid:
    def __init__(self):
        #self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.grid = np.zeros((GRID_SIZE, GRID_SIZE))
        self.walks = np.zeros((GRID_SIZE, GRID_SIZE))        
        self.zones = np.zeros((NOCLUSTERS, NOCLUSTERS))
        self.items = []
    
    @staticmethod
    def latToCellIndex(lat):
        return math.floor(lat*GRID_RESOLUTION) - CORNER_LAT_POS
    
    @staticmethod
    def lngToCellIndex(lng):
        return math.floor(lng*GRID_RESOLUTION) - CORNER_LNG_POS
    
    @staticmethod
    def cellIndexToLat(cell):
        return (cell + CORNER_LAT_POS + 0.5) / GRID_RESOLUTION
    
    @staticmethod
    def cellIndexToLng(cell):
        return (cell + CORNER_LNG_POS + 0.5) / GRID_RESOLUTION 
    
    @staticmethod
    def getClusterCenters():
        centers = []
        for i in range(NOCLUSTERS):
            for j in range(NOCLUSTERS):
                centers.append([i*NOCLUSTERS + j, Grid.cellIndexToLat((i+0.5)*CLUSTERED_DIAMETER), Grid.cellIndexToLng((j+0.5)*CLUSTERED_DIAMETER)])
        return centers
    
    def pointToCell(self, point):
        lat = point['lat']
        lng = point['lng']
        return self.latToCellIndex(lat), self.lngToCellIndex(lng)
    
    def pointToCell2(self, point):
        [lng, lat] = point
        return self.latToCellIndex(lat), self.lngToCellIndex(lng)
    
    def addAllPoints(self, points):
        for point in points:
            x,y = self.pointToCell(point)
            self.grid[x][y] = 1
            
        for i in range(len(points)-1):
            p1 = points[i]
            p2 = points[i+1]
            
            x1,y1 = self.pointToCell(p1)
            x2,y2 = self.pointToCell(p2)
            
            steps = max(x2-x1, y2-y1) * 20
            
            tx, ty = x1, y1
            for j in range(steps):
                tx = tx + (x2-x1)/steps
                ty = ty + (y2-y1)/steps
                self.grid[math.floor(tx)][math.floor(ty)] = 1
                
    def addWalkPoints(self, walks):
        for walk in walks:
            points = walk['points']
            score = walk['score']
            
            print(walk)
            
            for point in points:
                x,y = self.pointToCell2(point)
                try:
                    self.walks[x][y] = self.walks[x][y] + score
                except:
                    continue
                    
            for i in range(len(points)-1):
                p1 = points[i]
                p2 = points[i+1]

                x1,y1 = self.pointToCell2(p1)
                x2,y2 = self.pointToCell2(p2)

                steps = max(x2-x1, y2-y1) * 2

                tx, ty = x1, y1
                for j in range(steps):
                    tx = tx + (x2-x1)/steps
                    ty = ty + (y2-y1)/steps
                    try:
                        self.walks[math.floor(tx)][math.floor(ty)] = self.walks[math.floor(tx)][math.floor(ty)] + score
                    except:
                        continue
        
    def splitToItems(self):
        items = []
        
        # horizontal items
        for i in range(GRID_SIZE):
            item = []
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 1:
                    item.append([i,j])
                else:
                    if len(item) > 1:
                        items.append(item)
                        for [a,b] in item:
                            self.grid[a][b] = 2
                        item = []
            if len(item) > 0:
                items.append(item)
                for [a,b] in item:
                    self.grid[a][b] = 2
                item = []
                
        # vertical items
        for j in range(GRID_SIZE):
            item = []
            for i in range(GRID_SIZE):
                if self.grid[i][j] == 1:
                    item.append([i,j])
                else:
                    if len(item) > 0:
                        items.append(item)
                        for [a,b] in item:
                            self.grid[a][b] = 2
                        item = []
            if len(item) > 0:
                items.append(item)
                for [a,b] in item:
                    self.grid[a][b] = 2
                item = []
        
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 2:
                    self.grid[i][j] = 1
        
        self.items = items
        print(len(items))
        
    def computeOverlapping(self):
        totalitems = len(self.items)
        q = queue.PriorityQueue(maxsize=int(totalitems * PERCENT_ITEMS))
        
        for item in self.items:
            itemscore = 0
            for [x,y] in item:
                itemscore = itemscore + self.walks[x,y]
            itemscore = itemscore / len(item) #### is this good ??????????
            q.put((-itemscore, item))
        
        for i in range(int(totalitems * PERCENT_ITEMS)):
            (_, item) = q.get()
            for [x,y] in item:
                self.grid[x][y] = 0.5
        
# create matrix for planned points
g = Grid()

In [7]:
# add planned routes

#g.addAllPoints(planned_points)
for route in planned_routes:
    g.addAllPoints(route)
g.splitToItems()

# plt.ioff()
# plt.matshow(g.grid, aspect='auto')
# plt.savefig('grid_map2.png', dpi=500)

52243


In [8]:
clusters = Grid.getClusterCenters()
print(len(clusters))
with open("clusters.csv", 'w') as file:
    for cluster in clusters:
        file.write(str(cluster[0]) + ',' + str(cluster[1]) + ',' + str(cluster[2]) + '\n')

1225


In [9]:
# add walk routes
with open("output.py", "r") as file:
    walk_routes = json.loads(file.read())

print(len(walk_routes))



70


In [ ]:
g.addWalkPoints(walk_routes)

{'points': [[49.1079533, 16.4032638], [49.1090047, 16.4091325], [49.1276193, 16.413542], [49.1410947, 16.3891554], [49.1410947, 16.3888764], [49.174279, 16.4105701], [49.1862953, 16.4142823], [49.1864884, 16.4142501], [49.1922176, 16.4246356], [49.2125809, 16.4070511], [49.2340064, 16.4309764], [49.2346716, 16.4281976], [49.2441988, 16.4303863], [49.2459583, 16.4271462], [49.2471492, 16.4254832], [49.2746687, 16.4374995], [49.2757952, 16.4367163], [49.2808056, 16.4398813], [49.3243003, 16.4575732], [49.3330336, 16.4382398], [49.3339241, 16.4383149], [49.3367136, 16.4368129], [49.3373358, 16.4366949], [49.3446636, 16.4265347], [49.3453288, 16.4271033], [49.345479, 16.4267707], [49.3480432, 16.4261162], [49.3532789, 16.4274466], [49.3634713, 16.4269423], [49.3659711, 16.4271033], [49.365778, 16.4261377], [49.3998849, 16.4075553], [49.4105065, 16.4081132], [49.4200552, 16.398747], [49.4269216, 16.4100659], [49.4365025, 16.4100766], [49.4480142, 16.4004984]], 'score': 311}
{'points': [[49.

{'points': [[49.106763, 16.4832101], [49.1073632, 16.4869273], [49.1157103, 16.4766383], [49.1185534, 16.4747393], [49.1498601, 16.4636564], [49.1532719, 16.4859831], [49.154377, 16.4882147], [49.1545916, 16.4888048], [49.1562653, 16.4947057], [49.15663, 16.5047264], [49.1727233, 16.5079236], [49.1766822, 16.5042973], [49.1858339, 16.5066147], [49.1981292, 16.5141356], [49.2154992, 16.5216136], [49.2186642, 16.519382], [49.222312, 16.5216565], [49.2230844, 16.5223753], [49.2241251, 16.525315], [49.2245758, 16.5260339], [49.2255306, 16.5272892], [49.2261851, 16.5294671], [49.2263997, 16.5324712], [49.2267966, 16.5326107], [49.2275476, 16.5319026], [49.2280948, 16.5338445], [49.2355514, 16.5281904], [49.2379546, 16.5290165], [49.297843, 16.5221071], [49.2979932, 16.5221608], [49.298315, 16.5221071], [49.315964, 16.4750612], [49.3374431, 16.4798784], [49.341166, 16.4845026], [49.3570769, 16.494137], [49.3821824, 16.4915729], [49.3823218, 16.4913368], [49.3864632, 16.4842558], [49.3901646,

{'points': [[49.1078874, 16.6228894], [49.1051638, 16.612283], [49.1056359, 16.6047621], [49.1060328, 16.6030669], [49.1064727, 16.6025949], [49.1089189, 16.602788], [49.1127384, 16.6030133], [49.1131997, 16.6006744], [49.1215467, 16.6003954], [49.1247118, 16.609751], [49.127394, 16.6145039], [49.1521776, 16.6215742], [49.1558146, 16.6252756], [49.1558576, 16.6254365], [49.1560936, 16.6262519], [49.1611576, 16.6259944], [49.1667151, 16.6232908], [49.1667581, 16.6234195], [49.1668761, 16.6233873], [49.1721332, 16.6232264], [49.1721976, 16.6242671], [49.1755557, 16.6220462], [49.1800511, 16.6214669], [49.1958547, 16.6158879], [49.1963053, 16.6160595], [49.2011225, 16.6141391], [49.2211211, 16.6294062], [49.2217648, 16.6291165], [49.3083143, 16.6221106], [49.3102026, 16.612004], [49.310503, 16.6118324], [49.3282914, 16.6024339], [49.3305767, 16.6078949], [49.3287849, 16.618098], [49.341445, 16.6373456], [49.3479466, 16.6349423], [49.3498671, 16.6354036], [49.3547809, 16.6395664], [49.3593

{'points': [[49.1264176, 16.4003778], [49.1332412, 16.3918376], [49.1336167, 16.400528], [49.1198838, 16.4242494], [49.1138864, 16.4684415], [49.1151738, 16.4707053], [49.112556, 16.4761448], [49.1139615, 16.4787841], [49.1073632, 16.4869273], [49.1112471, 16.5163994], [49.1109896, 16.553607], [49.1247118, 16.609751], [49.1281879, 16.6212952], [49.1277051, 16.6228938], [49.1341746, 16.6333222], [49.1352904, 16.6384828], [49.135226, 16.6403711], [49.1341853, 16.6518617], [49.1340995, 16.653825], [49.1345501, 16.6543293], [49.1343033, 16.6548443], [49.1338634, 16.6577089], [49.133724, 16.6582024], [49.1386592, 16.666013], [49.1367388, 16.6688025], [49.1390991, 16.6728151], [49.1200984, 16.7004311], [49.1152489, 16.7185843], [49.1176629, 16.7197001], [49.117738, 16.7263305], [49.118607, 16.7265558], [49.1186821, 16.7267811], [49.1193259, 16.7297316], [49.124819, 16.7321563], [49.1240717, 16.7422334]], 'score': 88}
{'points': [[49.1396248, 16.4013326], [49.1395926, 16.3969445], [49.1408694

{'points': [[49.2898929, 16.4014506], [49.290204, 16.3998842], [49.2813098, 16.3940156], [49.2759347, 16.4361048], [49.2761815, 16.4370167], [49.2785311, 16.4384222], [49.2799151, 16.446265], [49.2801297, 16.449033], [49.2812991, 16.452992], [49.2833805, 16.4549983], [49.288584, 16.4586782], [49.2883801, 16.459161], [49.2939699, 16.4723575], [49.294281, 16.4746428], [49.2957509, 16.501776], [49.2953753, 16.501615], [49.2949891, 16.504544], [49.2948389, 16.5044904], [49.2933476, 16.5141356], [49.2969203, 16.5265596], [49.2980146, 16.5437794], [49.2980897, 16.5438652], [49.29883, 16.5472984], [49.2963946, 16.5584993], [49.2931545, 16.5599477], [49.2879725, 16.5731549], [49.2903221, 16.5853751], [49.2927039, 16.5986896], [49.2923498, 16.5988398], [49.3000209, 16.6267025], [49.2917919, 16.6299748], [49.2910731, 16.6623008], [49.2907834, 16.6627622], [49.289335, 16.6627407], [49.2866099, 16.6668069], [49.2796147, 16.676141], [49.2799258, 16.6915476], [49.2798829, 16.6916656], [49.28087, 16.

{'points': [[49.4382513, 16.405195], [49.4542265, 16.4007211], [49.4568229, 16.4028776], [49.4572949, 16.4205801], [49.4623697, 16.4256656], [49.4548166, 16.4357722], [49.4657278, 16.4561355], [49.4611788, 16.4653301], [49.4623053, 16.4667785], [49.4516516, 16.5376532], [49.4316852, 16.5783048], [49.4315135, 16.5787768], [49.4281769, 16.5803754], [49.4258916, 16.5886259], [49.4254518, 16.5908146], [49.4240248, 16.5912437], [49.4183171, 16.5989685], [49.4092727, 16.6217673], [49.4076633, 16.6362083], [49.4068694, 16.6370666], [49.4075882, 16.6388047], [49.4078243, 16.6400492], [49.4081891, 16.6405964], [49.4083178, 16.6408432], [49.4086826, 16.6408753], [49.4088221, 16.6411865], [49.4084466, 16.6455317], [49.4096053, 16.6461432], [49.4072878, 16.6694248], [49.413532, 16.6934252], [49.4136822, 16.7355359], [49.4152594, 16.7348814], [49.4362261, 16.7476074]], 'score': 22}
{'points': [[49.4480142, 16.4004984], [49.4542265, 16.4007211], [49.4568229, 16.4028776], [49.4572949, 16.4205801], [4

In [ ]:

g.computeOverlapping()


In [ ]:

plt.ioff()
plt.matshow(g.grid, aspect='auto')
plt.savefig('grid_map3.png', dpi=500)